In [67]:
import pandas as pd
import numpy as np
import cv2
dataset_path = 'fer2013.csv'
image_size=(48,48)

# Imported the dataset
Load the dataset, used emotions column for target variable. The input is in the form of pixels.

In [68]:
def load_fer2013():
    data = pd.read_csv(dataset_path)
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'),image_size)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).as_matrix()
    return faces, emotions

# Preprocessing data
Convert the each pixel in the range of 0 to 1 by dividing each pixel by 255.

In [69]:
def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

In [70]:
faces, emotions = load_fer2013()
faces = preprocess_input(faces)

C:\Users\Aditya\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


# Split the data into test and train

In [71]:
from sklearn.model_selection import train_test_split
num_samples, num_classes = emotions.shape
xtrain, xtest,ytrain,ytest = train_test_split(faces, emotions,test_size=0.2,shuffle=True)

# Preparing the input for RF
Converted the 4 dimensional data into 2 dimensional data.

In [72]:
x=xtrain.shape[0]
y=xtrain.shape[1]*xtrain.shape[2]
xtrain.resize((x,y)) # a 2D array
print(xtrain)

[[-0.6784314  -0.29411763 -0.2235294  ... -0.94509804 -0.8901961
  -0.85882354]
 [-0.4352941   0.00392163  0.28627455 ...  0.77254903  0.6862745
   0.47450984]
 [-0.27843136  0.19215691  0.18431377 ... -0.5529412  -0.5529412
  -0.54509807]
 ...
 [ 0.69411767 -0.18431371 -0.8117647  ...  0.60784316  0.67058825
   0.69411767]
 [-0.46666664 -0.5529412  -0.54509807 ...  0.34901965  0.34901965
   0.3411765 ]
 [ 0.43529415  0.45098042  0.4039216  ... -0.5686275  -0.46666664
  -0.23921567]]


In [73]:
x=xtest.shape[0]
y=xtest.shape[1]*xtest.shape[2]
xtest.resize((x,y)) # a 2D array
print(xtest)

[[ 0.92156863  0.92941177  0.9137255  ...  0.85882354  0.8509804
   0.85882354]
 [ 0.6392157   0.6392157   0.67058825 ...  0.03529418  0.04313731
   0.02745104]
 [-0.78039217 -0.69411767 -0.5372549  ... -0.5686275  -0.5294118
  -0.6784314 ]
 ...
 [ 0.96862745  0.96862745  0.96862745 ... -0.08235294 -0.01176471
   0.00392163]
 [-0.9372549  -0.92156863 -0.92156863 ... -0.24705881  0.5137255
   0.34901965]
 [ 0.9137255   0.9137255   0.92941177 ... -0.23137254 -0.15294117
  -0.00392157]]


In [74]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics

# Fit the model

In [75]:
clf_rf = RandomForestClassifier()
clf_rf.fit(xtrain, ytrain)
predicted = clf_rf.predict(xtest)
acc_rf = accuracy_score(ytest, predicted)
print ("random forest accuracy: ",acc_rf)

C:\Users\Aditya\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


random forest accuracy:  0.1330454165505712


# Result and Description
Random forest considered to be highly accurate and robust because of the number of decision trees in the process. Does not suffer from overfitting problem. Main reason,takes the average of all the predictors and cancels bias
Classification on an input image is carried out by randomly extracting subwindows in the image and propagating scaled subwindows down each tree in the forest. Votes from each of the random trees for all sampled subwindows are aggregated. A simple voting scheme with majority ruling awards the corresponding class label

# Accuracy
The accuracy is 13.30%